# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [3]:
import torch.nn.functional as F
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet, self).__init__()  #nn.Module.__init__
        self.conv1 = nn.Conv2d(1, 10, 5) #1*28*28->10*24*24
        self.pool = nn.MaxPool2d(2,2)
        #pool ->10*12*12
        self.conv2 = nn.Conv2d(10, 20, 5) #20*8*8
        #pool ->20*4*4
        self.fc1 = nn.Linear(20*4*4,120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x))) #卷积，激励，池化
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 20*4*4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1) #important
   
model = SimpleNet()

# TODO:define loss function and optimiter
criterion = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters()) #优化方法


Next, we can start to train and evaluate!

In [4]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    model.train()
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        optimizer.zero_grad()   # 所有参数的梯度清零
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()         #即反向传播求梯度
        optimizer.step()        #调用optimizer进行梯度下降更新参数
                
        
   

100%|██████████| 468/468 [00:17<00:00, 27.25it/s]


#### Q5:
Please print the training and testing accuracy.

In [13]:
 # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for images, labels in tqdm(train_loader):
        output = model(images)
        
        #计算test——loss和结果正确的数 模板
        test_loss += F.nll_loss(output, labels, reduction='sum').item()  
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(labels.view_as(pred)).sum().item()

    print('train_average_loss = %0.2f%%'% (test_loss/len(train_loader.dataset)*100))
    print('train_accuracy = %0.2f%%' % (correct/len(train_loader.dataset)*100))
    
    model.eval()
    test_loss = 0
    correct = 0
    for images, labels in tqdm(test_loader):
        output = model(images)
        
        test_loss += F.nll_loss(output, labels, reduction='sum').item()  
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(labels.view_as(pred)).sum().item()
        
    print('test_average_loss = %0.2f%%'% (test_loss/len(test_loader.dataset)*100))
    print('test_accuracy = %0.2f%%' % (correct/len(test_loader.dataset)*100))
    
    
test()


  8%|▊         | 6/78 [00:00<00:01, 59.74it/s]

train_average_loss = 1.72%
train_accuracy = 99.25%


100%|██████████| 78/78 [00:01<00:00, 62.32it/s]

test_average_loss = 3.96%
test_accuracy = 98.62%
